In [1]:
import serial
import threading
import time

class Uart_Task_Manager:
    def __init__(self, uart_port, uart_baud_rate):
        self.uart_port = uart_port
        self.uart_baud_rate = uart_baud_rate
        self.serial_connection = None
        self.uart_thread = None
        self.callback_func = None
        
        self.uart_connect()
            
    def uart_connect(self, uart_port, uart_baud_rate):
        # close current port
        if self.uart_port != uart_port and self.serial_connection != None:
            self.serial_connection.close()
            self.serial_connection = None
        
        self.uart_port = uart_port
        self.uart_baud_rate = uart_baud_rate
        self.connect()
            
    def uart_connect(self):
        if self.serial_connection != None:
            print("Uart port already connected:", self.uart_port)
            return
        
        # try connect to new port
        try:
            self.serial_connection = serial.Serial(self.uart_port, self.uart_baud_rate)
            print("connected to serial port", self.uart_port)
        except:
            print("unable to connect to serial port", self.uart_port)
        
        
    def uart_event_handler(self, data):
        # print("recived uart data:")
        self.callback_func(data)
        
        # try:
        #     data_str = data.decode().strip()
        #     print(">", data_str)
        # except:
        #     print(" - uart_data can't parse to string")
        # need uart protocal here =========================================================
        # TB Finish
        # try:
        #     msg_type = data[:5].decode().strip()

        #     if (msg_type == "[CMD]"):
        #         command_handler(data)
        # except:
        #     print(" - Unable to retrive message_-type")
    
    # for uart protocal
    def uart_decoder(self):
        # TB Finish
        pass
    
    # for uart protocal
    def uart_encoder(self):
        # TB Finish
        pass
    
    def sent_data(self, data):
        self.serial_connection.write(data)
    
    def attack_callback(self, callback_func):
        self.callback_func = callback_func
        print(f"[Uart] Successfully attached callback function, {type(self.callback_func)}")
    
    def uart_listening_thread(self):
        print("=== Enter uart listening thread === ")
        # listen to urat port
        self.sent_data("[LOGOF]--".encode())
        while True:
            if self.serial_connection != None:
                try:
                    if self.serial_connection.in_waiting > 0:  # Check if there is data available to read
                        data = self.serial_connection.readline()  # Read and decode the data
                        self.uart_event_handler(data)
                except serial.SerialException as e:
                    print(f"Serial communication error: {e}")
                    # self.serial_connection = None
                except:
                    # fail to read line
                    pass
            else:
                # print("uart thread running")
                # time.sleep(0.1)
                # self.callback_func("hello from uart") # testing callback--------------------------------

                print("try to reconnect")
                self.uart_connect()
                time.sleep(1)
    
    def run(self):
        print("Starting uart thread")
        self.uart_thread = threading.Thread(target=self.uart_listening_thread, args=(), daemon=True)
        self.uart_thread.start()
        print("Started uart thread")

In [2]:
import socket
import threading
import time
from datetime import datetime

class Socket_Manager():
    def __init__(self, socket_port):
        self.PACKET_SIZE = 1024
        self.socket_port = socket_port
        self.socket_thread = None
        self.send_socket = None
        self.send_address = None
        self.read_socket = None
        self.read_address = None
        self.socket = None
        self.callback_func = None
        
        self.initialize_socket()
    
    def initialize_socket(self):
        tcp_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        tcp_socket.bind(('localhost', self.socket_port))
        # tcp_socket.settimeout(None)  # dont timeout when waiting for response
        
        print("udp port binded to port", self.socket_port) 
        self.socket = tcp_socket
        self.accept_connection()
    
    def accept_connection(self):
        self.socket.listen(2) # 2 space in queue is enough
        
        print(f"Listening on 'localhost':{self.socket_port} for connection")
        send_socket, send_address = self.socket.accept()
        read_socket, read_address = self.socket.accept()
        
        # ====== inital extra handshake to confim it belong to our project =======
        # TB Finish
        # ====== end of confirmation
        self.send_socket = send_socket
        self.send_address = send_address
        
        self.read_socket = read_socket
        self.read_address = read_address
        print(f"Connected with send:{send_address} read:{read_address}")
    
    def send_data(self, data):
        if self.send_socket != None:
            # socket.sento(bytes, addr)
            self.send_socket.sendall(data)
        else:
            print("No socket connected")
    
    def socket_handler(self, data):
        print("recived data from socket:")
        # need to split the recived data into corresponding messages ---------------------------------------------
        # all the data sended from client in a short period of time will be read at once
        # define sepcial "END_OF_Message" pattern to split it
        # call task_handler after split message
        # TB Finish
        print(f"recived \"{data.decode('utf-8')}\"")
        try:
            message = data.decode('utf-8')
            if "[GET]" in message:
                self.callback_func(data)
        except:
            pass
        # ^^^ is just testing version of passing data to net manager function -- TB Finish
        
    def task_handler(self, message):
        # decode message, execute this task
        # TB Finish
        pass
    
    def attack_callback(self, callback_func):
        self.callback_func = callback_func
        print(f"[Socket] Successfully attached callback function, {type(self.callback_func)}")
        

    def socket_listening_thread(self):
        print("=== Enter socket listening thread === ")
        # listen to socket
        while True:
            if self.read_socket != None:
                try:
                    data = self.read_socket.recv(PACKET_SIZE)
                    if(len(data) > 0):
                        self.socket_handler(data)
                    else:
                        time.sleep(0.1)               
                except socket.timeout:
                    print("timeout happened, no message back")
            else:
                # try reconnect --------------------------------------------------------------------------
                # TB Finish
                time.sleep(1)
                
    def run(self):
        print("Starting socket thread")
        self.socket_thread = threading.Thread(target=self.socket_listening_thread, args=(), daemon=True)
        self.socket_thread.start()
        print("Started socket thread")

In [3]:
from enum import Enum

def log_data_hist(data_type, data, time):
    try:
        with open(data_type + "_hist.txt", 'a') as log_file:
            # Write text followed by a newline to the log file
            log_file.write(str(data) + ", " + str(time) + '\n')
    except:
        print("Can't log data to log_file", data_type, data)

class Node_Status(Enum):
    Active = 1
    Idol = 2
    Disconnect = 3
    
class Node:
    def __init__(self, name, address, uuid):
        self.name = name
        self.address = address
        self.uuid = uuid
        
        self.status = Node_Status.Active
        self.data_historys = {}
        
        self.data_historys["GPS"] = deque()            # (gps_data, time)
        self.data_historys["Load_Cell"] = deque()      # (load_data, time)
        self.data_historys["Robot_Request"] = deque()  # (load_data, time)
        self.last_contact_time = datetime.now()
        
    def getData(self, data_type):
        if data_type not in self.data_historys:
            print(f"data type {data_type} not exist")
            return None
        
        return self.data_historys[data_type][-1][0]
    
    def storeData(self, data_type, data):
        if data_type not in self.data_historys:
            self.data_historys[data_type] = deque()
        
        time = datetime.now()
        self.data_historys[data_type].append((data, time))
        log_data_hist(data_type, data, time)
        
        # only kept latest 10 data in server
        if len(self.data_historys[data_type]) > 10:
            self.data_historys[data_type].popleft()
            
    def update_status(self, status):
        # need to implment how in detile the status affects Node's other function
        # TB Finish
        self.status = status
        

In [4]:
shared_node_list = []

class Network_Manager():
    def __init__(self):
        # Node list data struct
        # pull the node list from esp_module
        # create data structures
        global shared_node_list
        self.node_list = shared_node_list
        self.socket_sent = None
        self.uart_sent = None
    
    
    def add_nodes(self, name, address, uuid):
        node = Node(name, address, uuid)
        self.node_list.append(node)
    
    def run_on_current_thread(self):
        print("Net Manager start running")
        while True:
            # print("Net Manager still running...")
            time.sleep(1)
            
    def attack_callback(self, socket_sent, uart_sent):
        self.socket_sent = socket_sent
        self.uart_sent = uart_sent
        
    def callback_socket(self, data):
        print(f"[NetM] Triger callback from socket thread: {data}")
        # testing code -----------------------
        message = data.decode('utf-8')
        if "[GET]" in message:
            message += "[N]"
            print(" => pass message to uart")
            self.uart_sent(message.encode())
        # testing code -----------------------
            
        
    def callback_uart(self, data):
        print(f"[NetM] Triger callback from uart thread: {data}")
        # testing code -----------------------
        message = data.decode('utf-8')
        if "[GET]" in message:
            message += "[N]"
            print(" -> pass message to socket")
            self.socket_sent(message.encode())
        # testing code -----------------------
    
    

In [ ]:
import time

PACKET_SIZE = 1024
socket_port = 5001
port = '/dev/ttyUSB0' #'COM7'
baud_rate = 115200

# Main function
def main():
    # Initialize The serial port & socket
    # Do locks sync for the shared vairable (serial_connection, socket)
    uart_manager = Uart_Task_Manager(port, baud_rate)
    socket_manager = Socket_Manager(socket_port)
    net_manager = Network_Manager()
    socket_manager.attack_callback(net_manager.callback_socket)
    uart_manager.attack_callback(net_manager.callback_uart)
    net_manager.attack_callback(socket_manager.send_data, uart_manager.sent_data)
    
    # Initialize uart_thread
    uart_manager.run()
    socket_manager.run()
    net_manager.run_on_current_thread()

if __name__ == "__main__":
    main()
    print("exit---------------")

connected to serial port /dev/ttyUSB0
udp port binded to port 5001
Listening on 'localhost':5001 for connection
Connected with send:('127.0.0.1', 35850) read:('127.0.0.1', 35866)
[Socket] Successfully attached callback function, <class 'method'>
[Uart] Successfully attached callback function, <class 'method'>
Starting uart thread
=== Enter uart listening thread === 
Started uart thread
Starting socket thread
=== Enter socket listening thread === 
recived data from socket:
recived "[GET] GPS Data"
[NetM] Triger callback from socket thread: b'[GET] GPS Data'
 => pass message to uart
Started socket thread
Net Manager start running
recived data from socket:
recived "[GET] GPS Data"
[NetM] Triger callback from socket thread: b'[GET] GPS Data'
 => pass message to uart
[NetM] Triger callback from uart thread: b'[GET] GPS Data[N]; [ESP] confirm recived from uart; \n'
 -> pass message to socket
recived data from socket:
recived "[---]"
recived data from socket:
recived "[GET] GPS Data"
[NetM] T